# SqlLite Jupyter Notebook Experiments

See environment setup notes on ./README.md

This file is meant as a parallel to the queries run in notes/210829_query_notes.txt, but this notebook runs entirely using SqlLite and not PostgreSQL

NOTE: got this code rolling with examples from: https://mungingdata.com/sqlite/create-database-load-csv-python/

In [3]:
import pandas as pd
from pathlib import Path
import sqlite3
from IPython.display import display, HTML

In [4]:
#Make sure it exists
dbPath = Path('../data/sqlLiteCensusDb.db')
if (not Path.is_file(dbPath)):
    dbPath.touch()

conn = sqlite3.connect(dbPath)
c = conn.cursor() #We'll hang on to this cursor for later queries

censusDataDF = pd.read_csv('../data/sub-est2019_all.csv')

print("Dataframe of loaded census data:")
display(censusDataDF)

censusMainTableName = "all_2019"
# write the data to a sqlite table
censusDataDF.to_sql(censusMainTableName, conn, if_exists='replace', index = False)

print("Data loaded into sql")

Dataframe of loaded census data:


,sumlev,state,county,place,cousub,concit,primgeo_flag,funcstat,name,stname,...,popestimate2010,popestimate2011,popestimate2012,popestimate2013,popestimate2014,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019
0,40,1,0,0,0,0,0,A,Alabama,Alabama,...,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,...,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,...,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281
3,162,1,0,484,0,0,0,A,Addison town,Alabama,...,751,750,743,742,739,734,731,726,723,718
4,162,1,0,676,0,0,0,A,Akron town,Alabama,...,355,347,347,343,338,339,333,332,331,328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81429,157,56,43,99990,0,0,1,F,Balance of Washakie County,Wyoming,...,2782,2755,2734,2737,2690,2690,2650,2599,2554,2531
81430,50,56,45,0,0,0,0,A,Weston County,Wyoming,...,7198,7142,7077,7136,7138,7208,7220,6968,6924,6927
81431,157,56,45,56215,0,0,1,A,Newcastle city,Wyoming,...,3530,3506,3476,3483,3484,3513,3530,3409,3394,3397
81432,157,56,45,79125,0,0,1,A,Upton town,Wyoming,...,1098,1089,1081,1088,1095,1104,1105,1062,1057,1056


Data loaded into sql


In [5]:
print("How many rows are in the db:")
c.execute(f"SELECT count(*) FROM {censusMainTableName}").fetchall() 

How many rows are in the db:


[(81434,)]

In [6]:
# States that grew the most:
c.execute('''select sumlev, name, stname, popestimate2019, census2010pop, cast(popestimate2019 as integer)-cast(census2010pop as integer) as stephdiff from all_2019 where sumlev=40 order by stephdiff desc;''').fetchall() 

[(40, 'Texas', 'Texas', 28995881, '25145561', 3850320),
 (40, 'Florida', 'Florida', 21477737, '18801310', 2676427),
 (40, 'California', 'California', 39512223, '37253956', 2258267),
 (40, 'North Carolina', 'North Carolina', 10488084, '9535483', 952601),
 (40, 'Georgia', 'Georgia', 10617423, '9687653', 929770),
 (40, 'Washington', 'Washington', 7614893, '6724540', 890353),
 (40, 'Arizona', 'Arizona', 7278717, '6392017', 886700),
 (40, 'Colorado', 'Colorado', 5758736, '5029196', 729540),
 (40, 'Virginia', 'Virginia', 8535519, '8001024', 534495),
 (40, 'South Carolina', 'South Carolina', 5148714, '4625364', 523350),
 (40, 'Tennessee', 'Tennessee', 6829174, '6346105', 483069),
 (40, 'Utah', 'Utah', 3205958, '2763885', 442073),
 (40, 'Oregon', 'Oregon', 4217737, '3831074', 386663),
 (40, 'Nevada', 'Nevada', 3080156, '2700551', 379605),
 (40, 'Massachusetts', 'Massachusetts', 6892503, '6547629', 344874),
 (40, 'Minnesota', 'Minnesota', 5639632, '5303925', 335707),
 (40, 'Maryland', 'Maryland

In [7]:
# learned a lesson about data types

c.execute(f"select sumlev, name, stname, popestimate2019, popestimate2010, cast(popestimate2019 as integer)-cast(popestimate2010 as integer) as stephdiff from all_2019 order by stephdiff desc;").fetchall() 


[(40, 'Texas', 'Texas', 28995881, 25241971, 3753910),
 (40, 'Florida', 'Florida', 21477737, 18845537, 2632200),
 (40, 'California', 'California', 39512223, 37319502, 2192721),
 (40, 'North Carolina', 'North Carolina', 10488084, 9574323, 913761),
 (40, 'Georgia', 'Georgia', 10617423, 9711881, 905542),
 (40, 'Washington', 'Washington', 7614893, 6742830, 872063),
 (40, 'Arizona', 'Arizona', 7278717, 6407172, 871545),
 (40, 'Colorado', 'Colorado', 5758736, 5047349, 711387),
 (50, 'Maricopa County', 'Arizona', 4485414, 3825110, 660304),
 (50, 'Harris County', 'Texas', 4713325, 4107666, 605659),
 (40, 'South Carolina', 'South Carolina', 5148714, 4635649, 513065),
 (40, 'Virginia', 'Virginia', 8535519, 8023699, 511820),
 (40, 'Tennessee', 'Tennessee', 6829174, 6355311, 473863),
 (40, 'Utah', 'Utah', 3205958, 2775332, 430626),
 (40, 'Oregon', 'Oregon', 4217737, 3837491, 380246),
 (40, 'Nevada', 'Nevada', 3080156, 2702405, 377751),
 (157, 'Balance of Harris County', 'Texas', 1936869, 1575189, 3

In [8]:
c.execute("select sumlev, name, stname, popestimate2019, popestimate2010, popestimate2019-popestimate2010 as stephdiff from all_2019 where name like '%ownsend%' order by stephdiff desc;").fetchall() 

[(162, 'Port Townsend city', 'Washington', 9831, 9119, 712),
 (157, 'Port Townsend city', 'Washington', 9831, 9119, 712),
 (162, 'Townsend town', 'Delaware', 2659, 2054, 605),
 (157, 'Townsend town', 'Delaware', 2659, 2054, 605),
 (61, 'Townsend town', 'Massachusetts', 9506, 8974, 532),
 (162, 'Townsend city', 'Montana', 2152, 1903, 249),
 (157, 'Townsend city', 'Montana', 2152, 1903, 249),
 (162, 'Townsend city', 'Tennessee', 462, 420, 42),
 (157, 'Townsend city', 'Tennessee', 462, 420, 42),
 (61, 'Townsend town', 'Wisconsin', 991, 976, 15),
 (61, 'Townsend township', 'Ohio', 1576, 1621, -45),
 (61, 'Townsend township', 'Ohio', 1534, 1617, -83)]

In [9]:

# just testing with casts, works!
c.execute(f"SELECT count(*) FROM {censusMainTableName}").fetchall() 
select sumlev, name, stname, popestimate2019, popestimate2010, popestimate2019-popestimate2010 as stephdiff from all_2019 order by stephdiff desc;


SyntaxError: invalid syntax (2202094926.py, line 3)

In [ ]:
# trying to figure out sumlev codes for a town like PT:

c.execute(f"select sumlev, name, stname, popestimate2019, popestimate2010, popestimate2019-popestimate2010 as stephdiff from all_2019 where name like '%ownsend%';").fetchall() 

# 162 | Port Townsend city | Washington    |            9831 |            9119 |       712
# 157 | Port Townsend city | Washington    |            9831 |            9119 |       712
# so 162 or 157, lets go with 162 for now

[(162, 'Townsend town', 'Delaware', 2659, 2054, 605),
 (157, 'Townsend town', 'Delaware', 2659, 2054, 605),
 (61, 'Townsend town', 'Massachusetts', 9506, 8974, 532),
 (162, 'Townsend city', 'Montana', 2152, 1903, 249),
 (157, 'Townsend city', 'Montana', 2152, 1903, 249),
 (61, 'Townsend township', 'Ohio', 1576, 1621, -45),
 (61, 'Townsend township', 'Ohio', 1534, 1617, -83),
 (162, 'Townsend city', 'Tennessee', 462, 420, 42),
 (157, 'Townsend city', 'Tennessee', 462, 420, 42),
 (162, 'Port Townsend city', 'Washington', 9831, 9119, 712),
 (157, 'Port Townsend city', 'Washington', 9831, 9119, 712),
 (61, 'Townsend town', 'Wisconsin', 991, 976, 15)]

In [ ]:
# Getting to some real stuff here!  For towns under 10000, who lost the most population?
c.execute(f"select sumlev, name, stname, popestimate2019, census2010pop, popestimate2010, popestimate2019-popestimate2010 as stephdiff from all_2019 where popestimate2019 < 10000 and sumlev = 162 order by stephdiff asc;").fetchall() 

[(162, 'Paradise town', 'California', 4476, '26218', 26198, -21722),
 (162, 'Marianna city', 'Florida', 5803, '6102', 7564, -1761),
 (162, 'Tuskegee city', 'Alabama', 8142, '9865', 9837, -1695),
 (162, 'Indianola city', 'Mississippi', 9037, '10683', 10610, -1573),
 (162, 'Eden city', 'Texas', 1294, '2766', 2782, -1488),
 (162, 'Bennettsville city', 'South Carolina', 7730, '9069', 9105, -1375),
 (162, 'Quincy city', 'Florida', 6827, '7972', 8019, -1192),
 (162, 'Middlesborough city', 'Kentucky', 9084, '10334', 10197, -1113),
 (162, 'Burlington city', 'Colorado', 3140, '4254', 4251, -1111),
 (162, 'St. Albans city', 'West Virginia', 9918, '11044', 11025, -1107),
 (162, 'Osceola city', 'Arkansas', 6638, '7757', 7730, -1092),
 (162, 'Atmore city', 'Alabama', 9107, '10194', 10169, -1062),
 (162, 'Winona city', 'Mississippi', 3964, '5043', 5020, -1056),
 (162, 'Coffeyville city', 'Kansas', 9275, '10295', 10267, -992),
 (162, 'Parsons city', 'Kansas', 9477, '10500', 10463, -986),
 (162, 'Unio

In [ ]:

# change as a percentage????  No, because in 2010 some of those places were zero
c.execute(f"select sumlev, name, stname, popestimate2019, census2010pop, popestimate2010, (popestimate2019-popestimate2010)/popestimate2010 as stephdiff from all_2019 where sumlev = 162 and popestimate2010 > 0 order by stephdiff asc;").fetchall() 




[(162, 'Abbeville city', 'Alabama', 2560, '2688', 2699, 0),
 (162, 'Adamsville city', 'Alabama', 4281, '4522', 4500, 0),
 (162, 'Addison town', 'Alabama', 718, '758', 751, 0),
 (162, 'Akron town', 'Alabama', 328, '356', 355, 0),
 (162, 'Alabaster city', 'Alabama', 33487, '30352', 31209, 0),
 (162, 'Albertville city', 'Alabama', 21711, '21160', 21196, 0),
 (162, 'Alexander City city', 'Alabama', 14317, '14875', 14929, 0),
 (162, 'Aliceville city', 'Alabama', 2254, '2486', 2480, 0),
 (162, 'Allgood town', 'Alabama', 650, '622', 622, 0),
 (162, 'Altoona town', 'Alabama', 913, '933', 937, 0),
 (162, 'Andalusia city', 'Alabama', 8680, '9015', 9025, 0),
 (162, 'Anderson town', 'Alabama', 267, '282', 281, 0),
 (162, 'Anniston city', 'Alabama', 21287, '23106', 22932, 0),
 (162, 'Arab city', 'Alabama', 8383, '8050', 8095, 0),
 (162, 'Ardmore town', 'Alabama', 1463, '1194', 1203, 0),
 (162, 'Argo town', 'Alabama', 4348, '4071', 4093, 0),
 (162, 'Ariton town', 'Alabama', 739, '764', 763, 0),
 (16

In [ ]:
# Count the different sumlev types:
c.execute(f"select count(*) from all_2019 where sumlev = 162;").fetchall() 

# > 19502

[(19502,)]

In [ ]:
# Show a count of each sumlev type:

c.execute(f"select distinct sumlev, count(*) from all_2019 group by sumlev;").fetchall() 

#  sumlev | count 
# --------+-------
#      40 |    51  #States?
#      50 |  3142  #Counties?
#      61 | 21063
#      71 | 13839
#     157 | 23714
#     162 | 19502
#     170 |     8
#     172 |   115

[(40, 51),
 (50, 3142),
 (61, 21063),
 (71, 13839),
 (157, 23714),
 (162, 19502),
 (170, 8),
 (172, 115)]

In [ ]:
# Show all of the states (sumlev = 40) and descending growth rates ("100*" to show more percentagy looking numbers)
c.execute(f"select sumlev, name, stname, popestimate2019, census2010pop, popestimate2010, 100*(popestimate2019-popestimate2010)/popestimate2010 as stephdiff from all_2019 where sumlev = 40 order by stephdiff desc;").fetchall() 

[(40,
  'District of Columbia',
  'District of Columbia',
  705749,
  '601723',
  605226,
  16),
 (40, 'Utah', 'Utah', 3205958, '2763885', 2775332, 15),
 (40, 'Colorado', 'Colorado', 5758736, '5029196', 5047349, 14),
 (40, 'Texas', 'Texas', 28995881, '25145561', 25241971, 14),
 (40, 'Arizona', 'Arizona', 7278717, '6392017', 6407172, 13),
 (40, 'Florida', 'Florida', 21477737, '18801310', 18845537, 13),
 (40, 'Idaho', 'Idaho', 1787065, '1567582', 1570746, 13),
 (40, 'Nevada', 'Nevada', 3080156, '2700551', 2702405, 13),
 (40, 'North Dakota', 'North Dakota', 762062, '672591', 674715, 12),
 (40, 'Washington', 'Washington', 7614893, '6724540', 6742830, 12),
 (40, 'South Carolina', 'South Carolina', 5148714, '4625364', 4635649, 11),
 (40, 'Georgia', 'Georgia', 10617423, '9687653', 9711881, 9),
 (40, 'North Carolina', 'North Carolina', 10488084, '9535483', 9574323, 9),
 (40, 'Oregon', 'Oregon', 4217737, '3831074', 3837491, 9),
 (40, 'Delaware', 'Delaware', 973764, '897934', 899593, 8),
 (40, '

In [ ]:
# Show the total growth of the country using sums of the state data (and format the big numbers nicely):
# printf number formatting see: https://database.guide/format-numbers-with-a-comma-in-sqlite
c.execute(f"select count(sumlev), count(stname), printf('%,d', sum(popestimate2019)) as pop2019, printf('%,d', sum(cast(census2010pop as integer))) as pop2010 from all_2019 where sumlev = 40;").fetchall() 



[(51, 51, '328,239,523', '308,745,538')]

In [ ]:
# Look for towns between 7000 and 10000 that have decreased in size:
# NOTE: This query required some postgresql > sqllite REGEX conversion thinking
# there were two routes to take, introduce a user defined regex function OR do a number formatting 
# equality check to verify that a number could be like a number

#For introducing a user defined regexp function, see: https://stackoverflow.com/a/58495085/2242421
import re
conn.create_function('regexp', 2, lambda x, y: 1 if not x is None and not y is None and re.search(x,y) else 0)
# conn.create_function('regexp', 2, lambda x, y: 1 if re.search(x,y) else 0)
        # popestimate2010 REGEXP '^\\d+$' 

#For the printf digit comparison approach, see: https://stackoverflow.com/a/32528946/2242421

c.execute('''
    select sumlev, name, stname, popestimate2019, census2010pop, popestimate2010, 100*(popestimate2019-popestimate2010)/popestimate2010 as stephdiff 
    from all_2019 where 
        printf('%d', popestimate2010) = popestimate2010
        and popestimate2010 > 0 
        and sumlev = 162 
        and popestimate2019 > 7000
        and popestimate2019 < 10000 
        and stephdiff < 0 
    order by stephdiff desc;
''').fetchall() 


[(162, 'Russellville city', 'Alabama', 9737, '9830', 9863, -1),
 (162, 'Sheffield city', 'Alabama', 8901, '9039', 9042, -1),
 (162, 'Tuscumbia city', 'Alabama', 8461, '8423', 8594, -1),
 (162, 'Westlake Village city', 'California', 8217, '8270', 8305, -1),
 (162, 'Highland city', 'Illinois', 9834, '9919', 9967, -1),
 (162, 'Park City city', 'Illinois', 7438, '7570', 7581, -1),
 (162, 'Willowbrook village', 'Illinois', 8579, '8540', 8697, -1),
 (162, 'Plymouth city', 'Indiana', 9982, '10033', 10131, -1),
 (162, 'Tell City city', 'Indiana', 7211, '7272', 7339, -1),
 (162, 'Creston city', 'Iowa', 7713, '7834', 7824, -1),
 (162, 'Denison city', 'Iowa', 8244, '8298', 8334, -1),
 (162, 'Grinnell city', 'Iowa', 9116, '9218', 9222, -1),
 (162, 'Knoxville city', 'Iowa', 7168, '7313', 7291, -1),
 (162, 'Perry city', 'Iowa', 7676, '7702', 7757, -1),
 (162, 'Corbin city', 'Kentucky', 7202, '7304', 7308, -1),
 (162, 'Paris city', 'Kentucky', 9671, '8553', 9815, -1),
 (162, 'Bath city', 'Maine', 833

In [ ]:

c.execute(f"select sumlev, name, stname, popestimate2019, census2010pop, popestimate2010, 100*(popestimate2019-popestimate2010)/popestimate2010 as stephdiff from all_2019 where sumlev = 162 and popestimate2019 > 7000 and popestimate2019 < 10000 and stephdiff < 0 order by stephdiff desc;").fetchall() 

[(162, 'Russellville city', 'Alabama', 9737, '9830', 9863, -1),
 (162, 'Sheffield city', 'Alabama', 8901, '9039', 9042, -1),
 (162, 'Tuscumbia city', 'Alabama', 8461, '8423', 8594, -1),
 (162, 'Westlake Village city', 'California', 8217, '8270', 8305, -1),
 (162, 'Highland city', 'Illinois', 9834, '9919', 9967, -1),
 (162, 'Park City city', 'Illinois', 7438, '7570', 7581, -1),
 (162, 'Willowbrook village', 'Illinois', 8579, '8540', 8697, -1),
 (162, 'Plymouth city', 'Indiana', 9982, '10033', 10131, -1),
 (162, 'Tell City city', 'Indiana', 7211, '7272', 7339, -1),
 (162, 'Creston city', 'Iowa', 7713, '7834', 7824, -1),
 (162, 'Denison city', 'Iowa', 8244, '8298', 8334, -1),
 (162, 'Grinnell city', 'Iowa', 9116, '9218', 9222, -1),
 (162, 'Knoxville city', 'Iowa', 7168, '7313', 7291, -1),
 (162, 'Perry city', 'Iowa', 7676, '7702', 7757, -1),
 (162, 'Corbin city', 'Kentucky', 7202, '7304', 7308, -1),
 (162, 'Paris city', 'Kentucky', 9671, '8553', 9815, -1),
 (162, 'Bath city', 'Maine', 833